In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
print("This is Rathod Mayur || 19BCE221 || This is my 3rd practical of IRS")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# I this fetch_20newsgroups dataset for make prediction using CountVectorizer.
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


In [ ]:
corpus = ['This is the first document.',
          'This document is the second document.',
          'And this is the third one.',
          'Is this the first document?',]


In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(X)
vectorizer.get_feature_names_out()


In [ ]:
print(X.toarray())


In [ ]:
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(corpus)
print(vectorizer2.get_feature_names_out())
print("***Vector Repersentation***\n",X2.toarray())


In [ ]:
# This is the Example for sklearn datasets.
# In this is I perform CountVectorizer and then make Prediction.

# Create our vectorizer
vectorizer = CountVectorizer()

train = fetch_20newsgroups(subset='train',remove=('headers', 'footers', 'quotes'))
test = fetch_20newsgroups(subset='test',remove=('headers', 'footers', 'quotes'))

# Get the training vectors
vectors = vectorizer.fit_transform(train.data)

clf = MultinomialNB(alpha=.01)
clf.fit(vectors, train.target)

# Get the test vectors
vectors_test = vectorizer.transform(test.data)

# Predict and score the vectors
pred = clf.predict(vectors_test)
acc_score = metrics.accuracy_score(test.target, pred)
f1_score = metrics.f1_score(test.target, pred, average='macro')

print('Total accuracy classification score: {}'.format(acc_score))
print('Total F1 classification score: {}'.format(f1_score))


In [ ]:
txt = ["He is ::having a great Time, at the park time?",
       "She, unlike most women, is a big player on the park's grass.",
       "she can't be going"]


In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(txt)
print(vectorizer.get_feature_names_out())


In [ ]:
print(X.shape)
X.toarray()


In [ ]:
txt1 = ['His smile was not perfect', 'His smile was not not not not so perfect', 'she not sang', 'she was not perfect']
print ("The text: ", txt1)


In [ ]:
# This will print out a list of words used, and their index in the vectors
print('Vocabulary: ')
print(vectorizer.vocabulary_)


In [ ]:
tf = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')
txt_fitted = tf.fit(txt1)
txt_transformed = tf.fit_transform(txt1)
txt_transformed.toarray()


In [ ]:
# if we wanted to get the vector for one word:
print('Hot vector: ')
print(vectorizer.transform(['smile']).toarray())


In [ ]:
idf = tf.idf_
print(dict(zip(txt_fitted.get_feature_names(), idf)))

# We see that the tokens 'sang','she' have the most idf weight because 
# they are the only tokens that appear in one document only.
# The token 'not' appears 6 times but it is also in all documents, so its idf is the lowest


In [ ]:
rr = dict(zip(txt_fitted.get_feature_names(), idf))


In [ ]:
token_weight = pd.DataFrame.from_dict(rr, orient='index').reset_index()
token_weight.columns=('token','weight')
token_weight = token_weight.sort_values(by='weight', ascending=False)
token_weight 


In [ ]:
# Plot bar graph on this txt.
sns.barplot(x='token', y='weight', data=token_weight)            
plt.title("Inverse Document Frequency(idf) per token")
fig=plt.gcf()
fig.set_size_inches(10,5)
plt.show()


In [ ]:
# get feature names 
feature_names = np.array(tf.get_feature_names())
sorted_by_idf = np.argsort(tf.idf_)

print("Features with lowest idf:\n{}".format(feature_names[sorted_by_idf[:3]]))
print("\nFeatures with highest idf:\n{}".format(feature_names[sorted_by_idf[-3:]]))


In [ ]:
train_data  =  pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
train_data.head()


In [ ]:
train_data.describe()


In [ ]:
test_data=pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_data.head()


In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
import pickle
import string
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import roc_auc_score , accuracy_score , confusion_matrix , f1_score
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
def clean_comment(text):
    text = text.lower()
    text = re.sub('<.*?>','',text).strip() # remove html chars
    text = re.sub('\[|\(.*\]|\)','', text).strip() # remove text in square brackets and parenthesis
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation marks
    text = re.sub("(\\W)"," ",text).strip() # remove non-ascii chars
    text = re.sub('\S*\d\S*\s*','', text).strip()  # remove words containing numbers
    return text.strip()


In [ ]:
# First convert into string
train_data.comment_text = train_data.comment_text.astype(str)

# now clean the data using clean_comment.
train_data.comment_text = train_data.comment_text.apply(clean_comment)
train_data.comment_text.head()


In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer
import en_core_web_sm
nlp = en_core_web_sm.load()

snow_stemmer = SnowballStemmer(language='english')

stopwords = nlp.Defaults.stop_words
def apply_stemmer(text):
    words = text.split()
    sent = [snow_stemmer.stem(word) for word in words if not word in set(stopwords)]
    return ' '.join(sent)


In [ ]:
# Apply Stemming
train_data.comment_text = train_data.comment_text.apply(apply_stemmer)
train_data.comment_text.head()


In [ ]:
X = train_data.comment_text
y = train_data.drop(['id','comment_text'],axis = 1)


In [ ]:
x_train,x_test,y_train,y_test =  train_test_split(X,y,test_size = 0.2,random_state = 45)


In [ ]:
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',     
    analyzer='word',            
    token_pattern=r'\w{1,}',    
    ngram_range=(1, 3),         
    stop_words='english',
    sublinear_tf=True)

word_vectorizer.fit(x_train)    
train_word_features = word_vectorizer.transform(x_train)


In [ ]:
X_train_transformed = word_vectorizer.transform(x_train)
X_test_transformed = word_vectorizer.transform(x_test)


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.linear_model import LogisticRegression
seed=100

log_reg = LogisticRegression(C = 10, penalty='l2', solver = 'liblinear', random_state=seed)

# fit model
classifier_ovr_log = OneVsRestClassifier(log_reg)
classifier_ovr_log.fit(X_train_transformed, y_train)


In [ ]:
y_train_pred_proba = classifier_ovr_log.predict_proba(X_train_transformed)
y_test_pred_proba = classifier_ovr_log.predict_proba(X_test_transformed)


In [ ]:
def make_test_predictions(df,classifier):
    df.comment_text = df.comment_text.apply(clean_comment)
    df.comment_text = df.comment_text.apply(apply_stemmer)
    X_test = df.comment_text
    X_test_transformed = word_vectorizer.transform(X_test)
    y_test_pred = classifier.predict_proba(X_test_transformed)
    return y_test_pred


In [ ]:
y_pred=make_test_predictions(test_data,classifier_ovr_log)
y_pred


In [ ]:
y_pred_df = pd.DataFrame(y_pred,columns=y.columns)
y_pred_df


In [ ]:
submission_df = pd.concat([test_data.id, y_pred_df], axis=1)
submission_df.to_csv('submission.csv', index = False)
